In [97]:
from sqlalchemy import Column, Integer, Numeric, String, Date, Index, Text, text, Float, create_engine  # The architects of our database schema, crafting the structure of our data haven.
from sqlalchemy.orm import sessionmaker, declarative_base  # Craftsmen who build the bridge between our Pythonic realm and the database underworld.
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.orm import declarative_base, sessionmaker

from rapidframes import RapiDF, QueryBuilder

import threading
import warnings
from typing import List, Dict
import polars as pl
import pandas as pd

In [ ]:
!pip3 install -r requirements.txt

In [98]:
import sys
import os

In [99]:
sys.path.append('/home/ubuntu/OTB/rapidframes/')
sys.path.append('/home/ubuntu/OTB/BMAPS/routes')

In [100]:
import datetime
from collections import namedtuple
from datetime import datetime, timedelta
from time import ctime
from typing import Dict, List
import pandas as pd
import numpy as np
import polars as pl
from rapidframes import QueryBuilder, RapiDF
from warnings import simplefilter

In [101]:
STORAGE_URL = 'lmdb://databases/bmaps_data?map_size=50GB'
rf = RapiDF(STORAGE_URL)

[2024-03-10 08:58:58.534] [rapidframes] [warning] LMDB path at /home/ubuntu/AP/databases/bmaps_data/ has already been opened in this process which is not supported by LMDB. You should only open a single RapiDF instance over a given LMDB path. To continue safely, you should delete this RapiDF instance and any others over the LMDB path in this process and then try again. Current process ID=[911332]


In [102]:
rf.get_library("RA_DATA").list_symbols()

[]

In [103]:
LIBRARY_NAME = "RA_DATA"

In [104]:
class PostgresDatabaseSecretsHolder:
    # Configuring the warehouse to store the contraband.
    USERNAME = "postgres"
    PASSWORD = "6eA1Wyp6l'sj"
    HOST     = "35.154.160.43"
    PORT     = "5432"
    DATABASE = "bmaps"

# Establish a connection to the MariaDB and Postgre Server, like making an untraceable phone call to an old friend.
# Connecting to the MariaDB and Postgre Instances is akin to reaching out to a contact in a foreign land.
# The connection stringv, a secret formula for establishing a clandestine rendezvous.
# mariadb_connection_string = f"mysql+pymysql://{MariaDBDatabaseSecretsHolder.USERNAME}:{MariaDBDatabaseSecretsHolder.PASSWORD}\
# @{MariaDBDatabaseSecretsHolder.HOST}:{MariaDBDatabaseSecretsHolder.PORT}/{MariaDBDatabaseSecretsHolder.DATABASE}"

postgres_connection_string = f"postgresql+psycopg2://{PostgresDatabaseSecretsHolder.USERNAME}:{PostgresDatabaseSecretsHolder.PASSWORD}\
@{PostgresDatabaseSecretsHolder.HOST}:{PostgresDatabaseSecretsHolder.PORT}/{PostgresDatabaseSecretsHolder.DATABASE}"

# Create the engines, the heart of our operation, silent yet powerful.
# mariadb_engine = create_engine(mariadb_connection_string)
postgres_engine = create_engine(postgres_connection_string)
# The blueprint of our data structure, meticulous and precise.
Base = declarative_base()
Session = sessionmaker(bind=postgres_engine)
session = Session()

class SalesDataTrnx(Base):
    __tablename__ = 'salesdata_trnx'
    ID               = Column(Integer, primary_key=True, autoincrement=True)
    INVENTLOCATIONID = Column(String(25))
    ITEMID           = Column(String(50))
    SALESQTY         = Column(Numeric(15, 3))
    LINEDISC         = Column(Numeric(15, 3))
    LINEAMOUNT       = Column(Numeric(15, 3))
    SALESPRICE       = Column(Numeric(15, 3))
    COSTPRICE        = Column(Numeric(15, 3))
    INVOICEDATE      = Column(Date())
    channel          = Column(String(50))
    family           = Column(String(30))

    __table_args__ = (
        Index('salesdata_trnx_INVOICEDATE_IDX', 'INVOICEDATE'),
        Index('salesdata_trnx_ITEMID_IDX', 'ITEMID', 'INVENTLOCATIONID', 'channel', 'family'),
        Index('salesdata_trnx_LINEAMOUNT_IDX', 'LINEAMOUNT'),
        Index('salesdata_trnx_INVENTLOCATIONID_1_IDX', 'INVENTLOCATIONID', 'ITEMID', 'channel'),
        Index('salesdata_trnx_INVENTLOCATIONID_2_IDX', 'INVENTLOCATIONID', 'channel', 'INVOICEDATE', 'ITEMID', 'family'),
        Index('salesdata_trnx_family_IDX', 'family', 'ITEMID', 'LINEAMOUNT'),
        Index('salesdata_trnx_family_amt_IDX', 'family', 'LINEAMOUNT'),
    )

class RATransactionTable(Base):
    __tablename__ = 'ra_transaction_table'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    Channel = Column(Text)
    country = Column(Text)
    Region = Column(Text)
    area = Column(Text)
    city = Column(Text)
    StoreId = Column(Integer)
    Store = Column(Text)
    season = Column(Text)
    Department = Column(Text)
    Family = Column(Text)
    SubFamily = Column(Text)
    Category = Column(Text)
    DOM_COMM = Column(Text)
    SubCategory = Column(Text)
    ExtendedSubCategory = Column(Text)
    Supplier = Column(Text)
    SubCategorySupplier = Column(Text)
    AssemblyCodeNickName = Column(Text)
    Status = Column(Text)
    ENDOFLife = Column(Text)
    Description = Column(Text)
    ItemLookupCode = Column(Text)
    BudgetYear = Column(Float)
    BDate = Column(Date)
    BudgetAmount = Column(Float)
    UnitsBuyBySku = Column(Numeric)
    BudgetGrossMarginP = Column(Float)
    BudgetCost = Column(Float)
    BudgetQTY = Column(Numeric)
    OpeningStock = Column(Numeric)
    ClosingStock = Column(Numeric)
    StockReceivedQty = Column(Integer)
    TotalPurchaseQty = Column(Numeric)
    StockOnHandQty = Column(Numeric)
    CurrentStockCost = Column(Float)
    budget_quarter = Column(Text)
    budget_month = Column(Text)
    budget_week = Column(Text)
    budget_day = Column(Text)

In [105]:
def fetch_item_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)
    print(f"[INFO] FETCH ITEM_MASTER TRANSACTIONS")
    
    ps_session = sessionmaker(bind=postgres_engine)
    ps_conn    = ps_session()
    item_data    = ps_conn.execute(text('''SELECT "ItemLookupCode" AS "ITEMID",
                                                 MAX("Description") AS "Description",
                                                 MAX("Department") AS "Department",
                                                 MAX("CategoryName") AS "Category",
                                                 MAX("Family") AS "Family",
                                                 MAX("subFamily") AS "SubFamily",
                                                 MAX("SubCategory") AS "SubCategory",
                                                 MAX("ExtendedSubCategory") AS "ExtendedSubCategory",
                                                 MAX("SubCategorySupplier") AS "SubCategorySupplier",
                                                 MAX("AssemblyCodeNickname") AS "AssemblyCodeNickName",
                                                 MAX("ENDOFLife") AS "ENDOFLife",
                                                 MAX("DOM_COMM") AS "DOM_COMM",
                                                 MAX("Status") AS "Status",
                                                 MAX("Supplier") AS "Supplier"
                                                 FROM 
                                                    itemmaster
                                                 GROUP BY 
                                                    "ItemLookupCode"'''))
    item_master      = pd.DataFrame(item_data, columns=item_data.keys())
    item_master["ENDOFLife"]  = item_master["ENDOFLife"].fillna("")
    item_master['ITEMID'] = item_master['ITEMID'].astype(str)

    lib.write("item_master", item_master, prune_previous_versions=True) # , prune_previous_versions=True
    return lib
lib = fetch_item_data()

[INFO] FETCH ITEM_MASTER TRANSACTIONS


In [106]:
q = QueryBuilder()
item_master = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('item_master', query_builder=q).data)
type(item_master.head(2))

polars.dataframe.frame.DataFrame

In [107]:
def fetch_otb_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)

    # Fetch RA Transactions
    print(f"[INFO] FETCH OTB TABLE TRANSACTIONS")
    
    ps_session = sessionmaker(bind=postgres_engine)
    ps_conn    = ps_session()
    otb_data    = ps_conn.execute(text("SELECT * from otb_table ;"))
    otb_df      = pd.DataFrame(otb_data, columns=otb_data.keys())

    otb_df["Budget_date"]      = pd.to_datetime(otb_df["Budget_date"])
    otb_df["ENDOFLife"]  = otb_df["ENDOFLife"].fillna("")
    # cols_to_conv        = ["ClosingStock", "TotalPurchaseQty", "StockOnHandQty", "UnitsBuyBySku", "BudgetQTY", "OpeningStock"]
    # otb_df[cols_to_conv] = otb_df[cols_to_conv].apply(pd.to_numeric)
    otb_df = otb_df.fillna(0)
    lib.write("otb_trnx", otb_df, prune_previous_versions=True)
    otb_df = None
    return lib
    
lib = fetch_otb_data()

[INFO] FETCH OTB TABLE TRANSACTIONS


In [108]:
q = QueryBuilder()
otb_trnx = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('otb_trnx', query_builder=q).data)
otb_trnx.head(2)

ITEMID,Description,Department,Category,Family,SubFamily,SubCategory,ExtendedSubCategory,SubCategorySupplier,AssemblyCodeNickName,ENDOFLife,DOM_COMM,Status,channel,INVENTLOCATIONID,Store,area,Region,budget_amount,budget_cost,budget_qty,budget_quarter,budget_week,budget_month,Budget_date,budget_year,budget_day,Supplier,otb_amount,otb_cost,otb_qty,stock_qty_actual,stock_qty_ly,stock_qty_lly
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,datetime[ns],str,str,str,f64,f64,f64,f64,f64,f64
"""1BLNLPI7330SI7…","""Lenovo 330S i7…","""BMOBILE Lenovo…","""Laptop""","""LAPTOP""","""BMOBILE""","""LAPTOP""","""LAPTOP""","""LAPTOP-LENOVO""","""1BLNLPI7330SI7…","""""","""B.MOBILE""","""No""","""B2B""","""Re-8001""","""Corporate Sale…","""Goods under bi…","""CAIRO""",0.0,0.0,0.0,"""1""","""09""","""March""",2024-03-09 00:00:00,"""2024""","""6""","""LENOVO""",0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
# Fetch CST Transactions
def fetch_cst_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)

    print(f"[INFO] FETCH CURRENT SALES TRANSACTIONS")
    
    ps_session            = sessionmaker(bind=postgres_engine)
    ps_conn               = ps_session()
    cst_data              = ps_conn.execute(text("SELECT * from current_sales_trnx;"))
    cst_df                = pd.DataFrame(cst_data, columns=cst_data.keys())
    cst_df["INVOICEDATE"] = pd.to_datetime(cst_df["INVOICEDATE"])
    cols_to_conv          = ["LINEAMOUNT", "SALESQTY", "COSTPRICE", "gross_sales", "historical_year"]
    cst_df[cols_to_conv]  = cst_df[cols_to_conv].apply(pd.to_numeric)
    str_cols = ['channel', 'history_quarter']
    cst_df= cst_df[str_cols].fillna('unknown')
    cst_df = cst_df.fillna(0)
    lib.write("cst_trnx", cst_df, prune_previous_versions=True)
    cst_df = None
    return lib

lib = fetch_cst_data()

[INFO] FETCH CURRENT SALES TRANSACTIONS


In [110]:
q = QueryBuilder()
cst_trnx  = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('cst_trnx', query_builder=q).data)

In [111]:
cst_trnx['history_quarter'].value_counts()

history_quarter,count
str,u32
"""unknown""",2
"""4""",1809122
"""2""",1732044
"""3""",1791936
"""1""",2135290


In [112]:
def fetch_stock_data():
  lib = rf.get_library('RA_DATA', create_if_missing=True)

  # Get the opening stock
  print(f"[INFO] FETCH STOCK TRANSACTIONS")

  # Get the opening stock
  ps_session = sessionmaker(bind=postgres_engine)
  ps_conn    = ps_session()

  stock_data    = ps_conn.execute(text('''	SELECT "ItemLookupCode" AS "ITEMID" ,
                                          "StoreID" AS "INVENTLOCATIONID",
                                          SUM("Quantity") AS "stock_on_hand_qty",
                                          SUM("Quantity") AS "opening_stock",
                                          SUM("Quantity") AS "closing_stock",
                                          AVG("Price") AS "current_stock_cost_at_retail",
                                          0 AS "stock_received_qty",
                                          SUM("Cost") AS "opening_stock_at_cost",
                                          0 AS "adjustment_at_cost",
                                          0 AS "adjustment_at_retail"
                                          FROM stock 
                                          GROUP BY "ItemLookupCode","StoreID";
                                    '''
                                  ))

  stock_df      = pd.DataFrame(stock_data, columns=stock_data.keys())
  cols_to_conv        = ["stock_on_hand_qty", "opening_stock", "closing_stock", "current_stock_cost_at_retail"]
  stock_df[cols_to_conv] = stock_df[cols_to_conv].apply(pd.to_numeric)
  stock_df = stock_df.fillna(0)
  lib.write("stock_trnx", stock_df,prune_previous_versions=True)
  stock_df = None
  return lib

lib = fetch_stock_data()

[INFO] FETCH STOCK TRANSACTIONS


In [113]:
q = QueryBuilder()
stock_trnx = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('stock_trnx', query_builder=q).data)
stock_trnx.head()

ITEMID,INVENTLOCATIONID,stock_on_hand_qty,opening_stock,closing_stock,current_stock_cost_at_retail,stock_received_qty,opening_stock_at_cost,adjustment_at_cost,adjustment_at_retail
str,str,f64,f64,f64,f64,i64,f64,i64,i64
"""1AIFIX0SW0SWTO…","""Re-1037""",30000.0,30000.0,30000.0,0.0,0,0.0,0,0
"""1STCIRST404010…","""Re-1006""",2.0,2.0,2.0,0.0,0,0.0,0,0
"""1BSAASMCVIVOY3…","""Re-1026""",3.0,3.0,3.0,0.0,0,0.0,0,0
"""ACCOUNTOPENFRE…","""Re-1041""",30000.0,30000.0,30000.0,0.0,0,0.0,0,0
"""1SCAARSTHLB10B…","""Re-1030""",1.0,1.0,1.0,0.0,0,0.0,0,0


In [114]:
def fetch_kpi_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)

    # Get the opening stock
    print(f"[INFO] FETCH KPI TRANSACTIONS")

    ps_session = sessionmaker(bind=postgres_engine)
    ps_conn    = ps_session()

    kpi_data    = ps_conn.execute(text("SELECT * from kpi_table;"))
    kpi_df      = pd.DataFrame(kpi_data, columns=kpi_data.keys())
    kpi_df = kpi_df.fillna(0)
    # ra_df["Budget_date"]      = pd.to_datetime(ra_df["Budget_date"])
    # ra_df["ENDOFLife"]  = ra_df["ENDOFLife"].fillna("")
    # cols_to_conv        = ["budget_qty"]

    # cols_to_conv        = ["ClosingStock", "TotalPurchaseQty", "StockOnHandQty", "UnitsBuyBySku", "BudgetQTY", "OpeningStock"]
    # ra_df[cols_to_conv] = ra_df[cols_to_conv].apply(pd.to_numeric)

    lib.write("kpi_trnx", kpi_df,prune_previous_versions=True)
    ra_df = None
    return lib
lib = fetch_kpi_data()

[INFO] FETCH KPI TRANSACTIONS


In [115]:
q = QueryBuilder()
kpi_trnx  = pl.DataFrame(rf.get_library(LIBRARY_NAME).read("kpi_trnx", query_builder=q).data)
kpi_trnx.head(2)

ITEMID,article_score_sale,article_score_abc,article_score_ae,article_score_speed,article_score_terminal,article_score_margin,article_score_sell,article_score_markdown,article_score_core,article_score_quartile,article_score_sortimeter,btech_vs_sortimeter,store_sku_count,price
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1BNOOMSMNKC102…",0.000562,0.000749,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""1MWPWDFSWPWD01…",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
rf.delete_library('SP_DATA')
rf.list_libraries()

['RA_DATA']

In [117]:
print(*map(lambda item:(print(item), item)[1], lib.list_versions().items()), 'the symbol version')

(stock_trnx_v0, (date=2024-03-10 09:00:49.199295371+00:00))
(otb_trnx_v0, (date=2024-03-10 08:59:05.109940682+00:00))
(kpi_trnx_v0, (date=2024-03-10 09:00:51.051610209+00:00))
(item_master_v0, (date=2024-03-10 08:59:03.567305919+00:00))
(cst_trnx_v0, (date=2024-03-10 09:00:35.987665714+00:00))
(stock_trnx_v0, (date=2024-03-10 09:00:49.199295371+00:00)) (otb_trnx_v0, (date=2024-03-10 08:59:05.109940682+00:00)) (kpi_trnx_v0, (date=2024-03-10 09:00:51.051610209+00:00)) (item_master_v0, (date=2024-03-10 08:59:03.567305919+00:00)) (cst_trnx_v0, (date=2024-03-10 09:00:35.987665714+00:00)) the symbol version


In [118]:
rf.get_library(LIBRARY_NAME).write(
            'ra_item_joined',
            otb_trnx.join(item_master,
            on=['ITEMID'],
            how='left' 
            ).to_pandas(), prune_previous_versions=True
        )

VersionedItem(symbol='ra_item_joined', library='RA_DATA', data=n/a, version=0, metadata=None, host='LMDB(path=/home/ubuntu/AP/databases/bmaps_data)')

In [128]:
q=QueryBuilder()
ra_item_joined = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('ra_item_joined', query_builder=q).data)
ra_item_joined=ra_item_joined.fill_null(0)

In [129]:
stock_trnx=stock_trnx.fill_null(0).fill_nan(0)

In [130]:
rf.get_library(LIBRARY_NAME).write(
            'ra_stock_joined',
            ra_item_joined.join(stock_trnx,
            on=['ITEMID'],
            how='left', 
            ).to_pandas(), prune_previous_versions=True
        )

[2024-03-10 09:02:23.410] [rapidframes] [info] Column INVENTLOCATIONID_right does not have non null elements.


VersionedItem(symbol='ra_stock_joined', library='RA_DATA', data=n/a, version=3, metadata=None, host='LMDB(path=/home/ubuntu/AP/databases/bmaps_data)')

In [131]:
q=QueryBuilder()
# ra_stock_joined = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('ra_stock_joined', query_builder=q).data)

ra_stock_joined = rf.get_library(LIBRARY_NAME).read('ra_stock_joined', query_builder=q).data
# ra_stock_joined.dtypes

pybind11::handle::inc_ref() is being called while the GIL is either not held or invalid. Please see https://pybind11.readthedocs.io/en/stable/advanced/misc.html#common-sources-of-global-interpreter-lock-errors for debugging advice.
If you are convinced there is no bug in your code, you can #define PYBIND11_NO_ASSERT_GIL_HELD_INCREF_DECREF to disable this check. In that case you have to ensure this #define is consistently used for all translation units linked into a given pybind11 extension, otherwise there will be ODR violations. The failing pybind11::handle::inc_ref() call was triggered on a NoneType object.


InternalException: std::runtime_error(pybind11::handle::inc_ref() PyGILState_Check() failure.)

In [53]:
rf.get_library(LIBRARY_NAME).write(
            'ra_kpi_joined',
            ra_stock_joined.join(kpi_trnx,
            on=['ITEMID'],
            how='left' 
            ).to_pandas(), prune_previous_versions=True
        )

VersionedItem(symbol='ra_kpi_joined', library='RA_DATA', data=n/a, version=0, metadata=None, host='LMDB(path=/home/ubuntu/AP/databases/bmaps_data)')

In [54]:
ra_kpi_joined  = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('ra_kpi_joined', query_builder=q).data)

In [ ]:
rf.get_library(LIBRARY_NAME).write(
        symbols.RA_SALES_JOIN,
        filtered_ra_trx.join(
            filtered_sales_trx,
            on=["ITEMID", "channel", "INVENTLOCATIONID"],
            how="left"
        ).collect().to_pandas()
    )